In [67]:
import pandas as pd
import os
import subprocess
import seaborn as sns
import kaggle

In [68]:
def created_folders():
    # Variavel responsável pos capiturar o path raiz do projeto
    path_folder = os.getcwd() 
    # Abrindo e carregando o arquivo CSV com os nomes das pastas que serão criadas para o projeto
    file_path = os.path.join(path_folder, 'nome_diretorios.csv')
    f = pd.read_csv(file_path)
    lista_folders = f['Folders'].to_list()

    #Criando o diretório "data"
    if not os.path.exists('data'):
            os.makedirs('data')

    # Laço for percorre a lista das partas cridas e inicia as pastas do projeto
    for i in lista_folders:
        diretorio = path_folder

        # Verifica se as pastas já foram criadas, se não, as cria.
        if not os.path.exists(diretorio + '/data/' + i):
            os.makedirs(diretorio + '/data/' + i)
            print(f"Diretório {i} criado com sucesso!")
        else:
            print(f"O diretório {i} já existe.")
created_folders()

O diretório engineer já existe.
O diretório raw já existe.
O diretório refined já existe.
O diretório transient já existe.
O diretório trusted já existe.


In [69]:
def download_csv_olist():
    folder = os.getcwd()
    path_folder = os.path.join(folder, 'data/transient')

    files = os.listdir(path_folder)

    if len(files) == 0:
        PATH_FOLDER = path_folder
        os.environ['PATH_FOLDER'] = PATH_FOLDER

        kaggle.api.dataset_download_files('olistbr/brazilian-ecommerce', path=PATH_FOLDER, unzip=True)
        print('Arquivos carregados e descompactados.')
    else:
        print()
        print(f'Arquivos já existem e ou foram atualizados!')
        print('Caso queira atualiza-los se faz necessário executar a recarga da Raw para zerar os arquivos.')
download_csv_olist()

Arquivos carregados e descompactados.


In [70]:
def created_arquivo_controller():
    folder = os.getcwd()
    path_folder = os.path.join(folder, 'data/transient')  # Substitua pelo caminho da pasta desejada
    file_path = os.path.join(folder, 'controller.csv')

    # Capturar nomes dos arquivos e remover a extensão
    arquivos = os.listdir(path_folder)

    # Eliminar as palavras indesejadas
    palavras_indesejadas = ['_dataset.csv', 'olist_', 'order_', '_dataset', 'product_', '_name', '_translation', '.csv']
    nomes_limpos = [arquivo for arquivo in arquivos]
    for palavra in palavras_indesejadas:
        nomes_limpos = [nome.replace(palavra, '') for nome in nomes_limpos]

    # Caminho do arquivo CSV a ser salvo
    caminho_arquivo = str(path_folder[0]) + '/' + 'controller.csv'

    # Definir os dados a serem escritos no arquivo CSV
    dados = [
        ["path_transient", "path_raw", "path_trusted", "table_transient", "table_raw", "table_trusted", "table_name", "table_name_temp"]
    ] + [
        [str(path_folder[0]) + "data/transient/", str(path_folder[0]) + "data/raw/", str(path_folder[0]) + "data/trusted/", nome.replace('.csv', ""), nome.replace('.csv', ""), nome.replace('.csv', ""), nome_limp, 'temp_' + nome_limp]
        for nome, nome_limp in zip(arquivos, nomes_limpos)
    ]

    # Salvar o arquivo CSV
    pd.DataFrame(dados).to_csv(file_path, index=False, header=False)
    print('Arquivo controller criado com sucesso!')
created_arquivo_controller()


Arquivo controller criado com sucesso!


In [71]:
def carregando_raw():
    folder = os.getcwd()
    file_path = os.path.join(folder, 'controller.csv')
    path = pd.read_csv(file_path)
    print('Path criado com base no arquivo controller!')

    for i in range(len(path['path_transient'])):
        # Criando o caminho dos arquivos e diretórios
        diretorio_transient = path['path_transient'][i]
        nome_arquivo_transient = path['table_transient'][i] + '.csv'
        caminho_arquivo_transient = (folder + diretorio_transient + nome_arquivo_transient)

        diretorio_raw = path['path_raw'][i]
        nome_arquivo_raw = path['table_raw'][i] + '.parquet'
        caminho_arquivo_raw = (folder + diretorio_raw + nome_arquivo_raw)

        # Verificando a existência dos arquivos em Transient e carrega na Raw pela primeira vez
        if os.path.exists(caminho_arquivo_transient) and not os.path.exists(caminho_arquivo_raw):
            # Carregando arquivos csv
            df = pd.read_csv(caminho_arquivo_transient)

            # Dropando os duplicados preservando o último atualizado
            df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)

            # Verifica se o DataFrame não está vazio
            if not df.empty:
                # Salva na Raw convertendo para o formato Parquet
                df.to_parquet(caminho_arquivo_raw)
                print(f"Arquivo Parquet {nome_arquivo_raw} criado: {caminho_arquivo_raw}")
            else:
                print(f"O DataFrame está vazio: {nome_arquivo_raw}")

        # Caso a Raw já tenha sido carregada pela primeira vez, esse código é executado para atualizar os arquivos verificando se algum arquivo foi deletado.
        elif os.path.exists(caminho_arquivo_transient) and os.path.exists(caminho_arquivo_raw):
            # Confere a existência do arquivo, caso deletado, cria novamente
            if not os.path.exists(caminho_arquivo_raw):
                df = pd.read_csv(caminho_arquivo_transient)
                df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)
                if not df.empty:
                    df.to_parquet(caminho_arquivo_raw)
                    print(f"Arquivo Parquet {nome_arquivo_raw} criado: {caminho_arquivo_raw}")

            # Carregando os DataFrames para concatenar os novos dados
            df1 = pd.read_csv(caminho_arquivo_transient)
            df2 = pd.read_parquet(caminho_arquivo_raw)

            # Concatenando os dados
            df_concatenado = pd.concat([df2, df1], axis=0)

            # Eliminando as duplicatas pelo ID mantendo sempre a última ocorrência
            df_concatenado = df_concatenado.drop_duplicates(subset=df_concatenado.columns[0], keep='last')
            df_concatenado.to_parquet(caminho_arquivo_raw)
        else:
            print(f'Arquivo Raw {nome_arquivo_raw} já existe! Para atualizá-lo, carregue a transient correspondente.')

    # Após a criação e atualização dos dados, os arquivos Transient são eliminados deixando o diretório
    # livre para receber novos dados
    print()
    print()
    for i in range(len(path['path_transient'])):
        diretorio_transient = path['path_transient'][i]
        nome_arquivo_transient = path['table_transient'][i] + '.csv'
        caminho_arquivo_transient = (folder + diretorio_transient + nome_arquivo_transient)
        
        if os.path.exists(caminho_arquivo_transient):
            os.remove(caminho_arquivo_transient)

        print(f"Arquivo {file_path} da Transient eliminado.")

carregando_raw()


Path criado com base no arquivo controller!


Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.
Arquivo /home/michel/Documentos/olist_brasil/controller.csv da Transient eliminado.


In [77]:
def carregando_trusted():
    folder = os.getcwd()
    file_path = os.path.join(folder, 'controller.csv')
    path = pd.read_csv(file_path)

    for i in range(len(path['path_raw'])):
        # Criando o caminho dos arquivos e diretórios
        diretorio_raw = path['path_raw'][i]
        nome_arquivo_raw = path['table_raw'][i] + '.parquet'
        caminho_arquivo_raw = (folder + diretorio_raw + nome_arquivo_raw)

        diretorio_trusted = path['path_trusted'][i]
        nome_arquivo_trusted = path['table_trusted'][i] + '.parquet'
        caminho_arquivo_trusted = (folder + diretorio_trusted + nome_arquivo_trusted)

        # Verificando a existência dos arquivos em Transient e carrega na Raw pela primeira vez
        if os.path.exists(caminho_arquivo_raw) and not os.path.exists(caminho_arquivo_trusted):
            # Carregando arquivos parquet
            df = pd.read_parquet(caminho_arquivo_raw)

            # Dropando os duplicados preservando o último atualizado
            df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)

            # Verifica se o DataFrame não está vazio
            if not df.empty:
                # Salva na Raw convertendo para o formato Parquet
                df.to_parquet(caminho_arquivo_trusted)
                print(f"Arquivo Parquet {nome_arquivo_trusted} criado: {caminho_arquivo_trusted}")
            else:
                print(f"O DataFrame está vazio: {nome_arquivo_trusted}")

        # Caso a Raw já tenha sido carregada pela primeira vez, esse código é executado para atualizar os arquivos verificando se algum arquivo foi deletado.
        elif os.path.exists(caminho_arquivo_raw) and os.path.exists(caminho_arquivo_trusted):
            # Confere a existência do arquivo, caso deletado, cria novamente
            if not os.path.exists(caminho_arquivo_trusted):
                df = pd.read_parquet(caminho_arquivo_raw)
                df.drop_duplicates(subset=df.columns[0], keep='last', inplace=True)
                if not df.empty:
                    df.to_parquet(caminho_arquivo_trusted)
                    print(f"Arquivo Parquet {nome_arquivo_trusted} criado: {caminho_arquivo_trusted}")

            # Carregando os DataFrames para concatenar os novos dados
            df1 = pd.read_parquet(caminho_arquivo_raw)
            df2 = pd.read_parquet(caminho_arquivo_trusted)

            # Concatenando os dados
            df_concatenado = pd.concat([df2, df1], axis=0)

            # Eliminando as duplicatas pelo ID mantendo sempre a última ocorrência
            df_concatenado = df_concatenado.drop_duplicates(subset=df_concatenado.columns[0], keep='last')
            df_concatenado.to_parquet(caminho_arquivo_trusted)
        else:
            print(f'Arquivo Raw {nome_arquivo_trusted} já existe! Para atualizá-lo, carregue a transient correspondente.')
carregando_trusted()
